In [1]:
import requests
#下載csv檔
csv_url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=datacreationdate desc&format=CSV'
res = requests.get(url=csv_url)
if res.ok:
    print("下載成功")
else:
    print("下載失敗")

下載成功


In [ ]:
import io
import csv
csv_str:str = res.text
csv_file = io.StringIO(csv_str)
dict_reader = csv.DictReader(csv_file)
csv_data = list(dict_reader)
for row in csv_data:
    print(row)


In [4]:
#檢查資料類型
#保証有資料
#過濾key

from pydantic import BaseModel,Field,field_serializer,RootModel
from datetime import datetime

class Site(BaseModel):
    站點名稱:str = Field(alias='sitename')
    城鎮:str = Field(alias='county')
    aqi:int
    狀態:str = Field(alias='status')
    日期:datetime = Field(alias='datacreationdate')

    @field_serializer("日期")
    def serialize_str(self,value):
        return value.strftime('%a %d %b %Y, %I:%M%p')

class Csvroot(RootModel):
    root:list[Site]

    def __iter__(self):
        return iter(self.root)

    def __getitem__(self, item):
        return self.root[item]

In [5]:
csv_root = Csvroot.model_validate(csv_data)
type(csv_root)

__main__.Csvroot

In [ ]:
for site in csv_root:
    print(type(site))

In [ ]:
for site in csv_root:
    print(site)

In [12]:
for site in csv_root:
    print(site.model_dump())

{'站點名稱': '屏東(枋山)', '城鎮': '屏東縣', 'aqi': 54, '狀態': '普通', '日期': 'Sun 17 Mar 2024, 01:00PM'}
{'站點名稱': '新北(樹林)', '城鎮': '新北市', 'aqi': 95, '狀態': '普通', '日期': 'Sun 17 Mar 2024, 01:00PM'}
{'站點名稱': '屏東(琉球)', '城鎮': '屏東縣', 'aqi': 97, '狀態': '普通', '日期': 'Sun 17 Mar 2024, 01:00PM'}
{'站點名稱': '臺南(麻豆)', '城鎮': '臺南市', 'aqi': 86, '狀態': '普通', '日期': 'Sun 17 Mar 2024, 01:00PM'}
{'站點名稱': '高雄(湖內)', '城鎮': '高雄市', 'aqi': 64, '狀態': '普通', '日期': 'Sun 17 Mar 2024, 01:00PM'}
{'站點名稱': '彰化(員林)', '城鎮': '彰化縣', 'aqi': 85, '狀態': '普通', '日期': 'Sun 17 Mar 2024, 01:00PM'}
{'站點名稱': '大城', '城鎮': '彰化縣', 'aqi': 116, '狀態': '對敏感族群不健康', '日期': 'Sun 17 Mar 2024, 01:00PM'}
{'站點名稱': '富貴角', '城鎮': '新北市', 'aqi': 66, '狀態': '普通', '日期': 'Sun 17 Mar 2024, 01:00PM'}
{'站點名稱': '麥寮', '城鎮': '雲林縣', 'aqi': 107, '狀態': '對敏感族群不健康', '日期': 'Sun 17 Mar 2024, 01:00PM'}
{'站點名稱': '關山', '城鎮': '臺東縣', 'aqi': 33, '狀態': '良好', '日期': 'Sun 17 Mar 2024, 01:00PM'}
{'站點名稱': '馬公', '城鎮': '澎湖縣', 'aqi': 50, '狀態': '良好', '日期': 'Sun 17 Mar 2024, 01:00PM'}
{'站點名稱': '金門', '城鎮': '金門縣'

In [21]:
with open('新aqi.csv',mode='w',encoding='utf-8',newline='') as file:
    dict_writer = csv.DictWriter(file,fieldnames=['站點名稱','城鎮','aqi','狀態','日期'])
    dict_writer.writeheader()
    for site in csv_root:
        dict_writer.writerow(site.model_dump())

print("存檔成功")

存檔成功


In [17]:
type(Site.model_fields)

dict

In [20]:
list(Site.model_fields.keys())

['站點名稱', '城鎮', 'aqi', '狀態', '日期']